# Physics Simulator

In [ ]:
# Python Standard Libraries
import logging
import math

# Numerical Python
import numpy as np

# Pandas
import pandas as pd

# Plotly 
import plotly.graph_objects as go
import plotly.express       as px
import plotly.subplots      as sp

#  Internal API
from tmns.math.rotations import Axis, Quaternion

In [ ]:
logging.basicConfig( level = logging.INFO )

## Vehicle Dynamics (Falcon 1)

In [ ]:
missile_radius_m = 0.25
mass_kg = 900

body_pitch_rad = 60 * math.pi / 180.0
body_yaw_rad   =-45 * math.pi / 180.0
body_roll_rad  =  0 * math.pi / 180.0

rocket_thrust = 107873.15 # Kilonewtons

rocket_thrust_acc = rocket_thrust / mass_kg # meters per second^2
logging.info( f'Rocket Acc: {rocket_thrust_acc}' )
lift_pct_of_g = 0.5

# Burn time
burn_time_s = 60

## Timing Characteristics


In [ ]:
t_0     =   0
t_max   = 500
t_delta =   1


## Setup Initial Launch Angle Orientation

In [ ]:
launch_quat = Quaternion.from_euler_angles( Axis.Y, body_pitch_rad,
                                            Axis.X, body_roll_rad,
                                            Axis.Z, body_yaw_rad )
launch_mat  = launch_quat.to_rotation_matrix()
display(launch_mat)

## Initial Position

In [ ]:
P_0 = np.array( [[0],[0],[0]], dtype = np.float64 )
display( P_0.T )

## Initial Velocity

In [ ]:
V_0 = np.array( [[0],[0],[0]], dtype = np.float64 )
display( V_0.T )

## Define Flight Characteristics

In [ ]:
launch_acc_body = np.array( [[39.2],
                             [ 0.0],
                             [ 0.0]], dtype = np.float64 )


## Physics Variables


In [ ]:
g_e = 9.807

A_g = np.array( [[0],[0],[-g_e]], dtype = np.float64 )

A_thrust_body = np.array( [[rocket_thrust_acc],
                           [ 0.0],
                           [ 0.0] ], dtype = np.float64 )

A_lift_body = np.array( [[0],[0],[lift_pct_of_g * g_e]], dtype = np.float64 )

res = { 'x':     [],
        'y':     [],
        'z':     [],
        'dist':  [],
        'pitch': [],
        'yaw':   [],
        't':     [] }


In [ ]:
def acc_from_drag( vel ):
    '''
    F_d = 0.5 * rho * v^2 * C_d
    '''

    # Coefficient for rocket
    C_d = 0.05
    
    # mass density of air
    rho = 1.2

    # Surface area
    A = math.pi * (missile_radius_m ** 2)
    return 0.5 * rho * vel * vel * C_d * A / mass_kg
    

In [ ]:
def velocity( dt, v_init, a_cur ):

    return v_init + a_cur * dt

In [ ]:
def position( dt, p_init, v_cur ):
    return p_init + v_cur * dt

In [ ]:
# Current elapsed time
t_cur = 0

# Working position and velocity information
P_init = P_0
V_init = V_0

P_cur = np.copy( P_init )
V_cur = np.copy( V_init )

# Iterate over each time step
while t_cur < t_max:

    # Compute body rotation matrix
    body_quat = Quaternion.from_euler_angles( Axis.Y, body_pitch_rad,
                                              Axis.X, body_roll_rad,
                                              Axis.Z, body_yaw_rad )

    # Compute thrust in the global coordinate system
    if t_cur < burn_time_s:
        A_thrust = body_quat.to_rotation_matrix() @ A_thrust_body
    else:
        A_thrust = np.zeros( (3,1), dtype = np.float64 )

    #  I want a glide vehicle, so I'll define lift here
    #A_lift = A_lift_body
    
    A_drag   = acc_from_drag( V_init )
    A_body   = A_thrust - A_drag

    # Accellerations
    A_cur = A_g + A_body # + A_lift

    # Compute Velocity
    V_cur = velocity( t_delta, V_init, A_cur )

    # Compute Position
    P_cur = position( t_delta, P_init, V_cur )

    # Create print string
    output =  f'Time: {t_cur}\n'
    output += f'    Position: {P_init.T} -> {P_cur.T}\n'
    output += f'    Velocity: {V_init.T} -> {V_cur.T}\n'
    output += f'    Accel:    {A_cur.T}\n'
    output += f'        Thrust: {A_thrust.T}\n'
    output += f'        Drag: {-A_drag.T}\n'
    output += f'        Grav: {A_g.T}\n'
    logging.debug( output )

    # Update body angles
    P_delta = P_cur - P_init
    base_dist = math.sqrt( P_delta[0][0] ** 2 + P_delta[1][0] ** 2 )
    eff_yaw   = math.atan2( P_delta[1][0], P_delta[0][0] )
    eff_pitch = math.atan2( P_delta[2][0], base_dist )

    total_dist = math.sqrt( P_cur[0][0] ** 2 + P_cur[1][0] ** 2 )
    
    # Update results
    res['x'].append( P_cur[0][0] )
    res['y'].append( P_cur[1][0] )
    res['z'].append( P_cur[2][0] )
    res['t'].append( t_cur )
    res['dist'].append( total_dist )
    res['pitch'].append( eff_pitch * 180.0 / math.pi )
    res['yaw'].append( eff_yaw * 180.0 / math.pi )

    if P_cur[2][0] < 0:
        print( f'Ground Collision Occurred at {t_cur} seconds' )
        break

    # Update time
    t_cur += t_delta

    # Update initial states
    P_init = np.copy( P_cur )
    V_init = np.copy( V_cur )

In [ ]:
df = pd.DataFrame( res )
df.head(5)


In [ ]:
fig = sp.make_subplots( rows = 6, cols = 1,
                        subplot_titles = [ 'Time vs Horizontal Distance',
                                           'Time vs Vertical Distance',
                                           'Horizontal vs Vertical Distance',
                                           'Pitch Angle',
                                           'Yaw Angle' ] )

fig.add_trace( go.Scattergl( x = df['t'],
                             y = df['dist'] ),
               row = 1, col = 1 )

fig.add_trace( go.Scattergl( x = df['t'],
                             y = df['z'] ),
               row = 2, col = 1 )

fig.add_trace( go.Scattergl( x = df['dist'],
                             y = df['z'] ),
               row = 3, col = 1 )

fig.add_trace( go.Scattergl( x = df['t'],
                             y = df['pitch'] ),
               row = 4, col = 1 )

fig.add_trace( go.Scattergl( x = df['t'],
                             y = df['yaw'] ),
               row = 5, col = 1 )

fig.add_vline( burn_time_s )
fig.update_layout( height = 1500 )
fig.show()